In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
# Returns a list of possible next coordinates
def get_candidate_moves(coord, visited):
    candidates = []
    
    for dx, dy in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
        new_coord = (coord[0] + dx, coord[1] + dy)
        if new_coord not in visited:
            candidates.append((dx, dy))
    return candidates


# Returns the coordinates visited in order of when they were visited
# and the moves made to get there, as well as whether the SAW is valid
def generate_loose_saw(L):
    coords = [(0, 0)]
    moves = []
    visited = set(coords)
    while len(coords) <= L:
        candidates = get_candidate_moves(coords[-1], visited)
        if len(candidates) == 0: # no possible moves, keep still
            coords += [coords[-1]]*(L+1-len(coords))
            break
        move = random.choice(candidates)
        coords.append((coords[-1][0] + move[0], coords[-1][1] + move[1]))
        moves.append(move)
        visited.add(coords[-1])
    return coords, moves, len(moves) == L


def generate_valid_saw(L):
    while True:
        coords, moves, valid = generate_loose_saw(L)
        if valid:
            return coords, moves


In [ ]:
generate_loose_saw(10)

([(0, 0),
  (-1, 0),
  (-1, -1),
  (-1, -2),
  (-2, -2),
  (-3, -2),
  (-3, -1),
  (-3, 0),
  (-2, 0),
  (-2, -1),
  (-2, -1)],
 [(-1, 0),
  (0, -1),
  (0, -1),
  (-1, 0),
  (-1, 0),
  (0, 1),
  (0, 1),
  (1, 0),
  (0, -1)],
 False)

Let $B(L_A, L_B)$ be the probability that when sampling SAWs of length $L_A$ and $L_B$ independently and uniformly at random, their concatenation is still a SAW.

$B(L_A, L_B) = {c_{L_A + L_B} \over c_{L_A} c_{L_B}}$

$\mu^{L_A} \approx {c_{L_A + L_B} \over c_{L_B}} = B(L_A, L_B) c_{L_A}$

We can generate samples of SAWs of length $L_A$ and $L_B$ to estimate $B(L_A, L_B)$

In [152]:
def estimate_B(L_A, L_B, n):
    Z = [(generate_valid_saw(L_A), generate_valid_saw(L_B)) for _ in range(n)]

    failures = 0
    for z_A, z_B in Z:
        visited = set(z_A[0])
        back_moves = z_B[1]
        curr_coord = z_A[0][-1]
        for move in back_moves:
            new_coord = (curr_coord[0] + move[0], curr_coord[1] + move[1])
            if new_coord in visited:
                failures += 1
                break
            else:
                curr_coord = new_coord
                visited.add(new_coord)

    return 1 - (failures / n)


# estimates c_{L_A + L_B}
def estimate_c(L_A, L_B, c_L_A, c_L_B, n):
    return estimate_B(L_A, L_B, n) * (c_L_A * c_L_B)


def estimate_mu(C):
    mus = []
    for c1, c2 in zip(C[:-1], C[1:]):
        mus.append((c2[1]/c1[1]) ** (1 / c1[0]))
    return mus


In [146]:
estimate_c(10, 10, 44100, 44100, 10_000)

826155287.9999999

In [149]:
C = [(10, 44100)]
n = 10_000
stopping_condition = 640 # stops when the last L is less than this value

while C[-1][0] < stopping_condition:
    L, c_L = C[-1]
    print(f'Working on L = {2*L}...')
    C.append((L*2, estimate_c(L, L, c_L, c_L, n)))

C

Working on L = 20...
Working on L = 40...
Working on L = 80...
Working on L = 160...
Working on L = 320...
Working on L = 640...


[(10, 44100),
 (20, 847353717.0),
 (40, 2.3478872120038512e+17),
 (80, 1.3450681439110571e+34),
 (160, 3.1462132541581814e+67),
 (320, 1.3056329691804971e+134),
 (640, 1.65694648160517e+267)]

In [153]:
estimate_mu(C)

[2.681406235260524,
 2.6432036452443666,
 2.6239265650403336,
 2.612841397591208,
 2.6083310177569987,
 2.6058439106974856]